# Encoder-Decoder Translation

In [ ]:
# !pip install transformers sentencepiece --quiet

# Mac install in terminal - conda install pytorch=2.6 torchvision torchaudio -c pytorch

# Windows install in terminal - conda install pytorch=2.6 torchvision torchaudio cpuonly -c pytorch

In [1]:
from transformers import pipeline
import torch

In [2]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
text = "The riverbank was covered with wildflowers."

/opt/anaconda3/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Helsinki-NLP/opus-mt-en-fr",
    output_hidden_states=True
)

inputs = tokenizer(text, return_tensors="pt")

# Get encoder hidden states only
encoder_outputs = model.model.encoder(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    output_hidden_states=True,
    return_dict=True
)

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [4]:
encoder_outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.2102, -0.2068, -0.0811,  ...,  0.1718,  0.3154, -0.0091],
         [ 0.2529, -0.2698, -0.4728,  ...,  0.2907, -0.4666,  0.1155],
         [-0.2714, -0.0846, -0.1218,  ...,  0.0221, -0.1134, -0.0924],
         ...,
         [-0.1508, -0.1977,  0.0820,  ...,  0.2332,  0.0769,  0.4241],
         [ 0.0443,  0.0423,  0.1285,  ...,  0.0590, -0.1956,  0.1630],
         [-0.0710, -0.0120, -0.0436,  ...,  0.0213, -0.1536,  0.0067]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=(tensor([[[-0.2734, -0.3387,  0.0982,  ..., -0.1762,  0.6778, -1.2438],
         [ 0.7630,  0.4338,  0.1135,  ...,  0.8364, -0.4221,  1.0364],
         [ 1.8897,  1.0854,  0.7504,  ..., -0.2146,  0.1899,  0.8723],
         ...,
         [ 1.6101,  0.5756,  0.9446,  ...,  0.2838,  0.4086,  0.8997],
         [ 0.3730,  0.5614,  0.7983,  ...,  0.9304, -0.1663,  0.5690],
         [-0.5201, -0.0647, -0.2032,  ...,  1.1537,  0.0423,  0.2740]]],
       grad_fn=<AddBack

In [5]:
for i, layer in enumerate(encoder_outputs.hidden_states):
    print(i, layer.shape)

0 torch.Size([1, 11, 512])
1 torch.Size([1, 11, 512])
2 torch.Size([1, 11, 512])
3 torch.Size([1, 11, 512])
4 torch.Size([1, 11, 512])
5 torch.Size([1, 11, 512])
6 torch.Size([1, 11, 512])


In [ ]:
translated_tokens = model.generate(
    inputs["input_ids"],
    max_length=40,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(translated_tokens[0], skip_special_tokens=True))
